In [7]:
import hashlib
import time


# Understanding Blockchain: Proof of Work and Merkle Tree

## Introduction
In this document, we'll explore the fundamental concepts of blockchain technology, focusing on the Proof of Work (PoW) consensus mechanism and the Merkle Tree data structure that underpins the blockchain's transactional integrity. We'll implement these concepts in Python and examine their role in maintaining the security and integrity of a blockchain.

## Proof of Work (PoW)
### Overview
Proof of Work (PoW) is a consensus mechanism used in blockchain networks to achieve agreement on the state of the network. It involves solving a computationally intensive puzzle to validate transactions and create new blocks in the blockchain. PoW is used in popular blockchain networks like Bitcoin and Ethereum.

### Explanation
Imagine you're a miner, and you want to add a block to the blockchain. The block includes a bunch of transactions, and your job is to find a special number called a "nonce" that, when combined with other block data, produces a specific pattern when hashed.

### Finding the Nonce
- You start with the block's data, including the transactions and a previous block's hash.
- You also pick a random number called a nonce and combine it with the block's data.
- Next, you use a computer to hash this combined data. A hash is like a fingerprint for data – it's unique to each set of data.
- If the resulting hash doesn't match the required pattern (usually starting with a certain number of zeros), you try another nonce and repeat the process.
- You keep trying different nonces until you find one that produces a hash with the required pattern.
### Why It's Important
The PoW puzzle is intentionally difficult and time-consuming to solve, but verifying the solution is easy. This ensures that adding a block to the blockchain requires real computational work, making it expensive for someone to tamper with the blockchain's history, but easy for others to verify the work.

### Implementation
We'll begin by implementing the PoW algorithm, including the Block and Blockchain classes.


In [8]:
class Block:
    def __init__(self, block_id, previous_hash, transactions, timestamp):
        self.block_id = block_id
        self.previous_hash = previous_hash
        self.transactions = transactions
        self.timestamp = timestamp
        self.nonce = 0

    def compute_hash(self):
        return hashlib.sha256((str(self.block_id) + self.previous_hash + str(self.transactions) + str(self.timestamp) + str(self.nonce)).encode()).hexdigest()

    def __str__(self):
        return f"\nBlock ID: {self.block_id}\nPrevious Hash: {self.previous_hash}\nTransactions: {self.transactions}\nTimestamp: {self.timestamp}\nNonce: {self.nonce}"


class Blockchain:
    def __init__(self):
        self.chain = []
        self.difficulty = 4  # Initial difficulty level
        self.target_time_interval = 15  # Target time interval between blocks in seconds
        self.block_count_since_adjustment = 0
        self.total_time_since_adjustment = 0
        # Create genesis block (initial block)
        genesis_block = Block(0, "0", "Genesis", time.time())
        self.add_block(genesis_block)

    def add_block(self, new_block):
        if len(self.chain) > 0:
            new_block.previous_hash = self.chain[-1].compute_hash()

        # Mine the block with adjusted difficulty
        new_block, time_taken = self.mine_block(new_block)

        self.chain.append(new_block)
        print("Block mined successfully.")
        print("Time taken:", time_taken, "seconds")

        self.block_count_since_adjustment += 1
        self.total_time_since_adjustment += time_taken

        # Adjust difficulty every multiple of 10 blocks
        if self.block_count_since_adjustment % 10 == 0:
            self.adjust_difficulty()

    def mine_block(self, block):
        start_time = time.time()
        target = "0" * self.difficulty

        while block.compute_hash()[:self.difficulty] != target:
            block.nonce += 1

        end_time = time.time()
        time_taken = end_time - start_time

        return block, time_taken

    def adjust_difficulty(self):
        average_time = self.total_time_since_adjustment / self.block_count_since_adjustment

        print("Average time taken to mine a block:", average_time, "seconds")

        buffer = 5  # Buffer time interval

        if average_time - 5 < self.target_time_interval:
            self.difficulty += 1
        elif average_time + 5 > self.target_time_interval:
            self.difficulty -= 1

        print("Difficulty adjusted to:", self.difficulty)

        # Reset counters
        self.block_count_since_adjustment = 0
        self.total_time_since_adjustment = 0

### Usage Example
We'll demonstrate the usage of the PoW algorithm by creating a blockchain and adding blocks to it.

In [9]:
blockchain = Blockchain()

# Create and add more blocks
for i in range(1, 100):
    print("\n---------Adding block #", i, "---------")
    new_block = Block(i, "", "Transaction data " + str(i), time.time())
    blockchain.add_block(new_block)
    print("Block #", i, " added to the blockchain", blockchain.chain[-1])

Block mined successfully.
Time taken: 0.03857564926147461 seconds

---------Adding block # 1 ---------
Block mined successfully.
Time taken: 0.08379721641540527 seconds
Block # 1  added to the blockchain 
Block ID: 1
Previous Hash: 0000e8bfe843e7a0c8063ccff95f84461317be7e343ead2da315b9337a8a0d49
Transactions: Transaction data 1
Timestamp: 1714873067.143669
Nonce: 60188

---------Adding block # 2 ---------
Block mined successfully.
Time taken: 0.04520106315612793 seconds
Block # 2  added to the blockchain 
Block ID: 2
Previous Hash: 00006a6bb7945775bf08379876fc998e4c3baedba7c626e5bb5a09493dd25e61
Transactions: Transaction data 2
Timestamp: 1714873067.2276547
Nonce: 32840

---------Adding block # 3 ---------
Block mined successfully.
Time taken: 0.10509753227233887 seconds
Block # 3  added to the blockchain 
Block ID: 3
Previous Hash: 0000bb0c9d538483915f3e7e2a13a253f3e94301e6810ebb62ea0e6ea6b07d1f
Transactions: Transaction data 3
Timestamp: 1714873067.272956
Nonce: 73999

---------Addin

## Merkle Tree


### Overview
The Merkle Tree is a data structure used to efficiently organize and verify the integrity of transactions within a block in a blockchain.

The Merkle Tree significantly reduces the data needed for proving the validity of a transaction in a blockchain.

In a blockchain, each block contains a large number of transactions. Without a Merkle Tree, verifying the inclusion of a transaction would require providing all the transactions in the block, which could be computationally expensive and inefficient, especially for large blocks.

However, with a Merkle Tree, the proof of inclusion for a transaction requires providing only a subset of hashes from the tree, known as the Merkle path. This path consists of hashes of sibling nodes along the path from the transaction's leaf node to the root of the tree.

The length of the Merkle path is logarithmic in the number of transactions in the block. Specifically, if a block contains 𝑛 transactions, the length of the Merkle path is approximately log₂(𝑛).

For example, if a block contains 1,000 transactions, the Merkle path would require providing only about 10 hashes (assuming a balanced Merkle Tree). This is a significant reduction compared to providing all 1,000 transactions.

Therefore, the Merkle Tree greatly reduces the amount of data needed for proving the validity of a transaction, making blockchain verification more efficient and scalable.

### Implementation
Next, we'll implement the Merkle Tree data structure, including functions for building the tree, retrieving Merkle proofs, and validating proofs.

### Usage Example
We'll demonstrate the usage of the Merkle Tree by constructing a tree from a list of transactions, retrieving a Merkle proof for a transaction, and validating the proof.